## Author: Farzan Memarian

In [6]:
# DATA GENERATION

import numpy as np
from itertools import permutations
import random
import time
from pdb import set_trace

Nexam = 10**5
Ndim = 20
Nperm = 30
x1 = np.random.multivariate_normal(mean= np.ones(Ndim), cov =  np.identity(Ndim),size = Nexam)
y1 = np.ones(Nexam)
x2 = np.random.multivariate_normal(mean= -np.ones(Ndim), cov =  np.identity(Ndim),size = Nexam)
y2 = -np.ones(Nexam)

X = np.concatenate((x1,x2),axis=0)
y = np.concatenate((y1,y2))

from sklearn import model_selection
X_tr_orig, X_test_orig, y_tr_orig, y_test_orig = model_selection.train_test_split(X,y,test_size=0.5)

# reshaping y
y_tr_orig = y_tr_orig.reshape((len(y_tr_orig),1))
y_test_orig = y_test_orig.reshape((len(y_test_orig),1))

perms = [] # array storing different premutatins of X, Y
for _ in range(Nperm):
    inx = np.random.permutation(Nexam)
    X_perm = X_tr_orig[inx]
    y_perm = y_tr_orig[inx]
    perms.append([X_perm,y_perm])

## a) Batch newton algorithm with the Gauss-Newton approximation

In [7]:
# FUNCTIONS
from numpy import outer, matmul, inner
from numpy.linalg import inv, norm
from scipy.sparse import diags
from sklearn.metrics import mean_squared_error

def func(X, theta):
    return  1.71 * np.tanh(0.66 * matmul(X, theta))

def f_prime(X, theta):
    return  1.71 * 0.66 *  (1 -  np.tanh(0.66 * matmul(X, theta)) )**2

def gradient_loss(f, f_prime, y, X):
    N = len(y)
    g = np.zeros((Ndim,))
    for i in range(N):
        nabla_f_theta = f_prime[i] * X[i,:]
        g += 0.5 * 2 * (f[i] - 1.5*y[i]) * nabla_f_theta
    return g.reshape((Ndim,1))

def hessian(f_prime, X):
    h = np.zeros((Ndim,Ndim))
    N,_ = np.shape(X)
    for i in range(N):
        h += f_prime[i]**2 * np.outer(X[i,:],X[i,:])
    return h

def batch_newton_step(X, y, theta):

    f = func(X, theta)
#     print ("mse f, y: {}".format(mean_squared_error(f,y)) )
    f_p = f_prime(X, theta)
    g = gradient_loss(f, f_p, y, X)
    h = hessian(f_p, X)
    h_inv = inv(h)
    d_theta = -matmul(h_inv, g)
    return d_theta

def batch_newton_iter(X, y, theta_init, thresh):
    theta = theta_init
    keep_iter = True
    counter = 0
    while keep_iter:
#         counter += 1
#         if counter % 10 == 0:
#             print ("iter:", counter)
#             print ("error", norm(d_theta))
#             print ("threshold", thresh)
        d_theta = batch_newton_step(X, y, theta)

        if norm(d_theta) > thresh:
            theta += d_theta
        else:
            keep_iter = False
    return theta



In [28]:
# TRAINING BATCH NEWTON ALGORITHM

# Nsizes = 5
# n_ex_float = np.floor(np.logspace(3.0, 5.0, num=Nsizes))
# n_ex = [int(item) for item in n_ex_float]
n_ex = [100,1000]

theta_store_all_N = []
time_storage_N = []
for N in n_ex:
    print ("RUNNING FOR {} EXAMPLES".format(N))
    start_time = time.time()
    theta_store = []
    thresh = 0.01/N
    perm_counter = 0
    for X_all,y_all in perms:
        X = X_all[:N,:]
        y = y_all[:N]
        perm_counter += 1
        if perm_counter % 5 == 0:
            print ("perm counter: ", perm_counter)
        theta_init = np.random.uniform(-0.5, 0.5, size=Ndim).reshape((Ndim,1))
        theta = batch_newton_iter(X, y, theta_init, thresh)
        theta_store.append(theta)
    end_time = time.time()
    elapsed_time = end_time - start_time
    time_storage_N.append(elapsed_time)
    theta_store_all_N.append(theta_store)

RUNNING FOR 100 EXAMPLES
perm counter:  5
perm counter:  10
perm counter:  15
perm counter:  20
perm counter:  25
perm counter:  30
RUNNING FOR 1000 EXAMPLES
perm counter:  5
perm counter:  10
perm counter:  15
perm counter:  20
perm counter:  25
perm counter:  30


In [24]:
# SAVE THETAS AND TIMES FOR NEWTON

import pickle

with open('theta_store_all_N.txt', 'wb') as fp:
    pickle.dump(theta_store_all_N, fp)
    
with open('time_storage_N.txt', 'wb') as fb:
    pickle.dump(time_storage_N, fb)

In [34]:
# FIND THETA* ON TEST SET USING NEWTON METHOD
        
N = len(y_test_orig)

thresh = 0.01/N

X = X_test_orig[:N,:]
y = y_test_orig[:N]

theta_init = np.random.uniform(-0.5, 0.5, size=Ndim).reshape((Ndim,1))
start_time = time.time()
theta_star = batch_newton_iter(X, y, theta_init, thresh)
np.save('theta_star', theta_star)
end_time = time.time()
elapsed_time_test = end_time - start_time
print ("elapsed time for test set: {}".format(elapsed_time_test))

elapsed time for test set: 222.98330950737


## B)  Implement the Online-Kalman algorithm 

In [16]:
# FUNCTIONS
def online_kalman(X,y,N):
    # Online Kalman Filter 

    phi = np.identity(Ndim) # Initializing as identity 
    theta = np.random.uniform(-0.5,0.5,size=Ndim) # Parameter vector has dimension Ndim 

    start_time = time.time()
    
    f = func(X, theta)
    df =  f_prime(X, theta)
    
    for j in range(N):
        tau = max(20,j-40)

        phi=np.linalg.inv((1-2/tau)*np.linalg.inv(phi)+ (2/tau)*(df[j]*df[j])*(np.outer(X[j,:],X[j,:])))

        dL=X[j,:]*(-2*df[j]*(1.5*y[j]-f[j]))

        theta = theta - (1/tau)*phi.dot(dL)
        
    time_o_run= (time.time() - start_time)

#     print("--- %s seconds ---" % (time_o_run))
    
    return (theta,time_o_run)

In [19]:
# TRAINING ONLINE KALMAN ALGORITHM

# Nsizes = 5
# n_ex_float = np.floor(np.logspace(3.0, 5.0, num=Nsizes))
# n_ex = [int(item) for item in n_ex_float]
n_ex = [5000,10000]

theta_store_all_K = []
time_storage_K = []
for N in n_ex:
    print ("RUNNING FOR {} EXAMPLES".format(N))
    
    theta_store = []
    thresh = 0.01/N
    perm_counter = 0
    start_time = time.time()
    for X_all,y_all in perms:
        X = X_all[:N,:]
        y = y_all[:N]
        perm_counter += 1
        if perm_counter % 5 == 0:
            print ("perm counter: ", perm_counter)
        theta, _ = online_kalman(X,y,N)
        theta_store.append(theta)
    end_time = time.time()
    elapsed_time = end_time - start_time
    time_storage_K.append(elapsed_time)
    theta_store_all_K.append(theta_store)

RUNNING FOR 5000 EXAMPLES
perm counter:  5
perm counter:  10
perm counter:  15
perm counter:  20
perm counter:  25
perm counter:  30
RUNNING FOR 10000 EXAMPLES
perm counter:  5
perm counter:  10
perm counter:  15
perm counter:  20
perm counter:  25
perm counter:  30


In [26]:
# SAVE THETAS AND TIMES FOR KALMAN
import pickle
    
with open('theta_store_all_K.plk', 'wb') as fp:
    pickle.dump(theta_store_all_K, fp)
    
with open('time_storage_K.plk', 'wb') as fb:
    pickle.dump(time_storage_K, fb)
    

[[array([  1.32389636e-03,   8.65723518e-01,   8.38047590e-01,
           7.15483447e-01,   3.19082799e-01,   1.93460201e+00,
          -8.82654368e-02,  -8.49958220e-01,  -3.82796728e-01,
           1.24129816e+00,   1.03346350e+00,  -1.16857533e+00,
          -7.42263373e-01,   8.70514706e-01,   1.62691873e+00,
           6.05269381e-01,  -5.55336138e-01,  -3.39992204e-01,
           7.91093397e-01,   1.06586355e+00]),
  array([ 0.14998666, -0.16896881, -0.25414899,  1.37774819,  0.629493  ,
          0.1713458 ,  0.23437769, -0.26944287,  0.60130454,  3.92160252,
         -0.84138704,  0.42334432,  2.3409012 ,  1.07291109,  0.06102787,
          1.33008374, -0.19104563,  0.51471738,  0.5663084 ,  0.02245444]),
  array([-0.07678538,  1.94463796,  1.30510681,  1.13129392,  0.37966976,
          0.47523911,  1.48520957, -0.41226545,  2.04564263,  2.95719831,
          0.35615637, -0.20641027,  1.42470928, -1.40113548, -0.15650203,
          0.42673488,  0.06261156, -2.02725919,  0.3203

## C) evaluations and figures

In [35]:
# EVALUATION OF METHODS ON TEST SET

# find mse error on test data
from sklearn.metrics import mean_squared_error
store_mse = []
X = X_test_orig
y = y_test_orig
for i, N in enumerate(n_ex):
    mse = 0
    for j in range(Nperm):
        theta = theta_store_all_N[i][j]
        f = func(X, theta)
        mse += mean_squared_error(f,y)/Nperm
    f_star = func(X, theta_star)
    mse_star = mean_squared_error(f_star,y)
    store_mse.append([mse,N])
print ("mse for different examples: ", store_mse)
print ("mse star: ", mse_star)



mse for different examples:  [[0.22935018849479444, 100], [0.23247181556807189, 1000]]
mse star:  0.233019854126


In [33]:
norm(theta_store_all_N[0][0] - theta_store_all_N[1][0])

0.36222003368403394

In [23]:
print ( np.mean([norm(theta_store_all[1][i] - theta_star) for i in range(Nperm)]))

0.0432198960423
